In [1]:
# Import Dependencies
import hvplot.pandas
import holoviews as hv
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,47.48,82,78,13.80,CL,1730176691
1,1,port-aux-francais,-49.3500,70.2167,36.79,82,96,20.00,TF,1730176519
2,2,keflavik,64.0049,-22.5624,41.43,87,75,24.16,IS,1730176694
3,3,margaret river,-33.9500,115.0667,73.09,57,15,15.26,AU,1730176695
4,4,weno,7.4515,151.8468,89.55,70,40,9.22,FM,1730176696


In [3]:
%%capture --no-display

vacation_map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, color = "City", alpha = 0.75, size = "Humidity", tiles = "OSM")

# Display the map
vacation_map_plot

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Vacation Spot Criteria: a max temp lower than 90, higher than 80, 
# cloudiness less than 2, wind speed less than 20 mph

city_data_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 90)
                                         & (city_data_df["Max Temp"] > 80) 
                                         & (city_data_df["Cloudiness"] < 2)
                                         & (city_data_df["Wind Speed"] < 20)
                                        ]


# Drop any rows with null values
city_data_filtered_df.dropna()

# Display sample data
city_data_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,saint-pierre,-21.3393,55.4781,80.28,69,0,14.97,RE,1730176748
250,250,loikaw,19.6742,97.2094,82.56,65,0,1.61,MM,1730176991
260,260,santana,-0.0583,-51.1817,80.78,78,0,8.05,BR,1730177004
297,297,mazatlan,23.2167,-106.4167,80.91,88,0,0.00,MX,1730177010
305,305,boa vista,2.8197,-60.6733,82.38,54,0,5.75,BR,1730177045
324,324,mukalla,14.5425,49.1242,80.87,72,0,5.26,YE,1730177084
328,328,sur,22.5667,59.5289,86.59,56,0,9.91,OM,1730177089
370,370,kish,26.5578,54.0194,86.27,39,0,11.50,IR,1730177137
401,401,key west,24.5557,-81.7826,80.60,74,0,13.80,US,1730176904
431,431,macapa,0.0389,-51.0664,80.58,78,0,8.05,BR,1730176923


In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
49,saint-pierre,RE,-21.3393,55.4781,69,
250,loikaw,MM,19.6742,97.2094,65,
260,santana,BR,-0.0583,-51.1817,78,
297,mazatlan,MX,23.2167,-106.4167,88,
305,boa vista,BR,2.8197,-60.6733,54,
324,mukalla,YE,14.5425,49.1242,72,
328,sur,OM,22.5667,59.5289,56,
370,kish,IR,26.5578,54.0194,39,
401,key west,US,24.5557,-81.7826,74,
431,macapa,BR,0.0389,-51.0664,78,


In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
            
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel was found!"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
loikaw - nearest hotel: Nawaday Hotel
santana - nearest hotel: Hotel Teodoro
mazatlan - nearest hotel: Mona Inn
boa vista - nearest hotel: Aipana Plaza Hotel
mukalla - nearest hotel: فندق الجابري
sur - nearest hotel: Sur Hotel
kish - nearest hotel: هتل فارابی
key west - nearest hotel: Harborside Hotel & Marina
macapa - nearest hotel: Equatorial Hotel
turbat - nearest hotel: No hotel was found!
monywa - nearest hotel: Hotel Chindwin
gunnedah - nearest hotel: The Court House
yanchep - nearest hotel: Yanchep Inn
geraldton - nearest hotel: Ocean Centre Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
49,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
250,loikaw,MM,19.6742,97.2094,65,Nawaday Hotel
260,santana,BR,-0.0583,-51.1817,78,Hotel Teodoro
297,mazatlan,MX,23.2167,-106.4167,88,Mona Inn
305,boa vista,BR,2.8197,-60.6733,54,Aipana Plaza Hotel
324,mukalla,YE,14.5425,49.1242,72,فندق الجابري
328,sur,OM,22.5667,59.5289,56,Sur Hotel
370,kish,IR,26.5578,54.0194,39,هتل فارابی
401,key west,US,24.5557,-81.7826,74,Harborside Hotel & Marina
431,macapa,BR,0.0389,-51.0664,78,Equatorial Hotel


In [7]:
%%capture --no-display

# remove rows for which no hotel was found because these shouldn't display in the map
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel was found!"]

# Configure the map plot
city_hotel_map_plot = hotel_df.hvplot.points("Lng", 
                                             "Lat", 
                                             geo = True,
                                             color = "City",
                                             alpha = 0.8,
                                             width = 600,
                                             height = 400,
                                             size = "Humidity",
                                             tiles = "OSM",
                                             hover_cols = ["Hotel Name", "Country"],
                                            )

# save the map to an external file
hv.save(city_hotel_map_plot, "output_data/City_Hotel_Map.html")

# Display the map
city_hotel_map_plot

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.